In [ ]:
#텍스트 마이닝 전처리를 위한 Mecab 모듈 임포트
from konlpy.tag import Mecab

# 텍스트 마이닝 학습시키는 gensim의 fasttext 모듈 임포트
from gensim.models import fasttext
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim import utils

import numpy as np
import pandas as pd
import tempfile
import os
import re
import sys
import time

# 파이썬 GUI 라이브러리 임포트
from PyQt5.QtWidgets import *
from PyQt5.Qt import Qt
from lxml import etree

# 미리 훈련된 모델 로드
model = fasttext.load_facebook_model("C:/Users/pranst/superbigmodel.bin", encoding="utf-8")

# 텍스트 마이닝 전처리를 하기 위한 Mecab 사전 파일 로드
mecab = Mecab("C:/mecab/mecab-ko-dic")

# 설문지별 키워드를 딕셔너리로 지정
keywords = {'2016년 대학교 금연사업 운영평가':['담배', '금연', '흡연'],
           '광주지역 노동환경 실태조사 설문지_근로자':['노동', '근무', '임금', '직장'],
           '설문지_전남대학교 대학원생 인권실태조사':['인권', '대학원'],
           '설문지_2018문화예술지원사업 만족도 조사':['공연', '전시', '문화', '예술'],
           '장애인문화예술실태조사_통합':['장애', '장애인', '예술'],
           '설문지_광주광역시 주요정책에 대한 시민 인식도 조사':['정책'],
           '설문지_2020년 광주광역시 자원봉사 실태조사':['자원봉사', '봉사'],
           '설문지_2020 광주 평생교육 만족도 조사_일반':['평생교육',' 교육']
           }

In [ ]:
# 대용량 말뭉치(corpus) 데이터셋을 사용하기 위해 Iterator로 관리하는 클래스
class MyIter:
    def __init__(surveyname):
        self.surveyname = surveyname # 설문지 이름을 받아와서
     
    def __iter__(self):
        path = datapath(self.surveyname) #해당 이름을 가진 설문지에서
        with utils.open(path, 'r', encoding='utf-8') as fin:
            for line in fin:
                re.sub('[^A-Za-z0-9]', '',line) #특수문자를 제거하고
                #re.sub('[^A-Za-z0-9가-힣]', '',line)
                yield list(mecab.morphs(line)) #단어 토큰으로 잘라 Iterator을 만든다.
                
# UI에서 "학습하기" 버튼을 눌렀을 때 pretrained 모델을 추가로 학습시키는 함수
def modelTraining(surveyName):
    model.build_vocab(MyIter(surveyName), update=True)# 대용량의 말뭉치를 학습시킬 수 있는 형태(vocab)으로 바꿔준다.
    model.train(MyIter(urveyName), total_examples =len(model.wv.key_to_index), epochs=1)
    # 추가로 학습시킨 새로운 모델을 덮어쓰기
    fasttext.save_facebook_model(model, "superbigmodel.bin")

In [ ]:
# 설문에 적합한 회원들을 추천하는 클래스
class processing:
    # 적합한 회원 몇명을 뽑아 화면에 출력하는 함수
    def extract(keywordlist, weight, interest=False, n=10, sample=5): # 설문지 키워드, 회원정보 항목별 가중치, 관심사를 계산에 포함시킬지 여부, 추천 알고리즘 계산에 사용되는 상수, 뽑을 회원수
        #회원정보 엑셀을 불러온다.
        df = pd.read_excel("C:/Users/pranst/20대이상회원정보.xlsx", sheet_name='시트0', usecols='F:L')

        # 계산된 similarity를 저장
        new_data= []
        
        # 추천 시 관심사를 사용할 것인가
        if interest:
            weight.append((4-sum(weight))/4)
        else:
            weight.append(0)

        #연관도 계산
        for i in range(len(df)):
            tmp = []

            for j in range(5):
                info = df.iloc[i][j]
                
                # 뒤에 '학과'또는 '공학과'가 붙어있을 경우 연관도가 많이 떨어지는 경우가 발생
                # 이를 방지하기 위해서 제거
                if j==1:
                    if info.rfind('공학과') != -1:
                        info = info[0:info.rfind('공학과')]
                    elif info.rfind('학과') != -1:
                        info = info[0:info.rfind('학과')]
                    elif info.rfind('과') != -1:
                        info = info[0:info.rfind('과')]
                
                
                for k in range(len(keywordlist)):
                    sim = model.wv.similarity(info, keywordlist[k])
                    # 이상치에 영향을 줄이기 위해
                    value = sim/(1+n*abs(sim))
                    tmp = tmp + [value]

            new_data.append(tmp)

        sdf = pd.DataFrame(new_data)

        data = pd.DataFrame()

        cn = ['전공', '세부전공', '직업', '세부직업', '관심사']

        # 위에서는 유저 정보와 설문지 키워드간의 연관도 결과를 각각의 열에 저장
        # 예를 들어 유저 정보가 5개, 키워드가 3개라면 총 15개의 열이 존재
        
        # 아래의 반복문을 통해서 이를 유저 정보의 수만큼으로 열을 줄임
        # 예를 들어 설문의 키워드가 3개라면 3개의 열을 더해서 하나의 열로 합침
        for i in range(5):
            c = sdf[i*len(keywordlist)]
            k = 1
            
            # 하나의 유저 정보와 모든 설문 키워드간의 연관도를 합함
            for j in range(len(keywordlist)-1):
                c += sdf[i*len(keywordlist)+k]
                k += 1

            data = pd.concat([data,c], axis=1)
        
        # 열 이름 설정
        data.columns = cn
        
        # 모든 행의 라벨을 우선 None으로 설정
        data['라벨'] = None
        
        #라벨링
        for i in range(len(data)):
            # 앞의 두 열은 전공과 세부 전공, 뒤의 두 열은 직업과 세부 직업이므로
            # 앞의 두 열의 합이 크다면 전공과 관련이 높아서 선정된 유저
            # 반대의 경우 직업과 연관이 높아서 선정된 유저임을 확인할 수 있음
            if (data.iloc[i][0] + data.iloc[i][1]) > (data.iloc[i][2] + data.iloc[i][3]):
                data.loc[i,'라벨'] = '전공'
            else:
                data.loc[i,'라벨'] = '직업'

        data['sum']=0
        
        for i in range(len(data)):
            value = 0
            
            # 각각의 유저 정보에 weight를 부여하여 유저와 설문간의 총 연관도를 계산
            # 예를 들어 weight가 [a,b,c,d,e]라면 sum = a*sim전공 + b*세부전공... 과 같이 계산
            for j in range(4):
                value += (weight[j] * data.iloc[i][j])

            data.loc[i,'sum'] = value       

        # 총 연관도가 가장 높은 n명을 선정(n은 parameter의 sample값)
        sorting_data = data.sort_values(by=['sum'], axis=0, ascending=False).head(sample)

        view = []
        #c = df.loc[data.sort_values(by=['sum'], axis=0, ascending=False).head(5).index[0]]
        
        # 선정된 n명에게 라벨을 붙임
        for i in range(5):
            if sorting_data.iloc[i]['라벨'] == '전공':
                c = df.loc[sorting_data.index[i]].drop('직업라벨링').tolist()
            elif sorting_data.iloc[i]['라벨'] == '직업':
                c = df.loc[sorting_data.index[i]].drop('전공라벨링').tolist()

            view.append(c)

        return view


In [ ]:
# GUI 함수(실제로 화면에 띄워지는 부분)
class InputKeywords(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.state = False
        self.n = 0
        self.sample = 0
        self.filename = ""
        self.setupUi()
    # UI 요소 배치 함수
    def setupUi(self):
        # 창 설정
        self.setWindowTitle("전문가 패널 추천 시스템") # 타이틀
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)
        layout = QGridLayout()
        self.centralWidget.setLayout(layout)      
        
        ## 1 번 그룹
        groupBox1 = QGroupBox("파일")
        layout.addWidget(groupBox1)
        
        Box1 = QHBoxLayout()
        groupBox1.setLayout(Box1)
        
        self.filenameText = QLineEdit()
        Box1.addWidget(self.filenameText)
        self.getFileBtn = QPushButton("찾아보기", self)
        self.getFileBtn.clicked.connect(self.getFileClicks)
        Box1.addWidget(self.getFileBtn)
        
        ## 2 번 그룹
        groupBox2 = QGroupBox("키워드")
        layout.addWidget(groupBox2)
        
        Box2 = QHBoxLayout()
        groupBox2.setLayout(Box2)
        
        self.keywordLabel = QLabel("키워드", self)
        Box2.addWidget(self.keywordLabel)
        self.keywordText = QLineEdit()
        Box2.addWidget(self.keywordText)
        
        ## 3 번 그룹
        groupBox3 = QGroupBox("옵션")
        layout.addWidget(groupBox3)
        
        Box3 = QHBoxLayout()
        groupBox3.setLayout(Box3)

        self.majorLabel = QLabel("전공", self)
        Box3.addWidget(self.majorLabel)
                                             
        self.majorText = QLineEdit('0.6')        
        Box3.addWidget(self.majorText)
        
        self.dmajorLabel = QLabel("세부전공", self)
        Box3.addWidget(self.dmajorLabel)
                                             
        self.dmajorText = QLineEdit('0.7')        
        Box3.addWidget(self.dmajorText)
        
        self.jobLabel = QLabel("직업", self)
        Box3.addWidget(self.jobLabel)
                                             
        self.jobText = QLineEdit('0.8')        
        Box3.addWidget(self.jobText)
        
        self.djobLabel = QLabel("세부직업", self)
        Box3.addWidget(self.djobLabel)
                                             
        self.djobText = QLineEdit('0.9')        
        Box3.addWidget(self.djobText)
        
        self.interestCk = QCheckBox("관심사", self)        
        self.interestCk.stateChanged.connect(self.setState)
        Box3.addWidget(self.interestCk)

        self.nLabel = QLabel("n", self)
        Box3.addWidget(self.nLabel)
                                             
        self.nText = QLineEdit('10')        
        Box3.addWidget(self.nText)
                                             
        self.sampleLabel = QLabel("sample", self)
        Box3.addWidget(self.sampleLabel)
                                             
        self.sampleText = QLineEdit('5')        
        Box3.addWidget(self.sampleText)
                                          
        ## 4 번 그룹
        groupBox4 = QGroupBox("버튼")
        layout.addWidget(groupBox4)
        
        Box4 = QHBoxLayout()
        groupBox4.setLayout(Box4)
            
        self.resetBtn = QPushButton("리셋", self)        
        self.resetBtn.clicked.connect(self.resetClicks)
        Box4.addWidget(self.resetBtn)
        
        self.resBtn = QPushButton("추천", self)
        self.resBtn.clicked.connect(self.resClicks)
        Box4.addWidget(self.resBtn)
        
        self.trainBtn = QPushButton("학습", self)
        self.trainBtn.clicked.connect(self.trainClicks)
        Box4.addWidget(self.trainBtn)
        
        ## 5번 그룹
        groupBox5 = QGroupBox("전문가 패널 추천 결과")
        layout.addWidget(groupBox5)
        
        Box5 = QHBoxLayout()
        groupBox5.setLayout(Box5)
        
        self.resTable = QTableWidget()
        self.resTable.setColumnCount(6)
        self.resTable.setRowCount(20)

        self.resTable.setHorizontalHeaderLabels(['전공', '세부전공', '직업', '세부직업', '관심사', '라벨'])
        
        Box5.addWidget(self.resTable)
    
    
    # 체크박스 상태 변경 함수
    def setState(self, state):
        if state == Qt.checked:
            self.state = True
        else:
            self.state = False  
    
    # 찾아보기 버튼 함수
    def getFileClicks(self):
        #파일 불러와서
        self.fname = QFileDialog.getOpenFileName(self, 'Open file', 'C:/Users/pranst/Desktop/캡스톤_설문/', 'File(*.txt)')
        #첫번째 선택한 파일의 경로에서 "파일이름.확장자" 만 빼온다
        #ex) C:\Users\user\a.txt   ->   a.txt
        if self.fname[0]:
            fname_list = self.fname[0].split('/')
            self.filename = fname_list[-1]
            self.filenameText.setText(self.filename)
    #리셋 버튼 함수
    def resetClicks(self):
        #파일이름리스트 초기화
        self.fname = []
        self.filename = ""
        
        # 텍스트 초기화
        self.filenameText.clear()
        self.keywordText.clear()
        
        # 표 초기화
        for i in range(20):
            for j in range(6):
                self.resTable.setItem(i, j, QTableWidgetItem(''))
        
        self.n = 0
        self.sample = 0
        
        # 각 항목별 가중치 입력
        self.majorText.setText('0.6')
        self.dmajorText.setText('0.7')
        self.jobText.setText('0.8')
        self.djobText.setText('0.9')
        
        self.nText.setText('10')
        self.sampleText.setText('5')
    
    # 결과 버튼 함수
    def resClicks(self):
        self.wlist = []
        #가중치 리스트를 생성
        self.wlist.append(float(self.majorText.text()))
        self.wlist.append(float(self.dmajorText.text()))
        self.wlist.append(float(self.jobText.text()))
        self.wlist.append(float(self.djobText.text()))        
        
        # 확장자를 뺀 파일이름만 가져와서 설문지 키워드 딕셔너리로부터 키워드 리스트를 불러온다.
        self.keywordlist = keywords[self.filename.replace('.txt', '')]
        
        # 키워드 "#" 표시 붙여서 화면에 출력
        self.keywordText.setText('#'+' #'.join(self.keywordlist))
        
        # 사용자 입력받은 n 값, sample 값을 정수값으로 바꿔서 n변수, sample변수에 저장
        self.n = int(self.nText.text())
        self.sample = int(self.sampleText.text())
        
        # 처리한 모드 변수들을 인자로 하여 추천 함수 실행
        self.list = processing.extract(self.keywordlist, self.wlist, self.state, self.n, self.sample)
        
        # PyQT 표로 추천 결과를 표시
        for i in range(len(self.list)):
            for j in range(len(self.list[0])):
                self.resTable.setItem(i, j, QTableWidgetItem(self.list[i][j]))
    # 학습 버튼 함수
    def trainClicks(self):
        #학습 시작 메시지
        startwarn = QMessageBox.warning(self, '알림', '학습이 시작되었습니다.')
        # 초세기 시작
        #self.timer.start()
        
        # 초세기 종료
        #startwarn.close()
        
        # 학습시작 메시지 ok 클릭하면 학습 시작
        if startmsg == QMessageBox.Ok:
            modelTraining(self.fname[0])
            
        #학습 종료 메시지
        endwarn = QMessageBox.warning(self, '알림', '학습이 완료되었습니다.')

In [ ]:
#메인 함수
def main():
    app = QApplication(sys.argv)
    win = InputKeywords() # GUI 함수 실행
    win.show() #GUI를 화면에 표시
    sys.exit(app.exec_()) #QApplication(GUI) 실행

#메인 함수를 실행시킨다.
if __name__ == '__main__':
    main()